In [130]:
from collections.abc import Mapping
from itertools       import product
from typing          import Generator, List
from helpers         import data

In [3]:
start = data(17)
start

['#.#####.',
 '#..##...',
 '.##..#..',
 '#.##.###',
 '.#.#.#..',
 '#.##..#.',
 '#####..#',
 '..#.#.##']

In [4]:
len(start), len(start[0])

(8, 8)

A **neighbor** is any cube where **any coordinate differs by at most 1**. 

Active cubes are denoted with \#, and inactive cubes are denoted with \. 

The boot process consists of six cycles. During each cycle: 
- If a cube is **active** and exactly **2 or 3 neighbors are active**, it stays active. Otherwise it becomes inactive. 
- If a cube is **inactive** and exactly **3 neighors are active**, it becomes active. Otherwise it stays inactive. 

**Part 1:** How many cubes are left in the active state after the sixth cycle?

In [172]:
def nested_get(d, *keys):
    if isinstance(d, Mapping):
        first, *rest = keys
        return nested_get(d[first], *rest)
    return d

In [173]:
class MutableBool:
    def __init__(self, val=False):
        self.val = val 
        self.next = False
        
    def __bool__(self):
        return self.val
    
    def __repr__(self):
        return str(self.val)
    
    def update(self):
        self.val, self.next = self.next, False

In [210]:
# At each time step, update each entry based on the rule and its neighbors 
class NGrid(dict):
    """NGrid will choose default values for missing keys and is nested n deep."""

    # Tracks minimum and maximum range for each level 
    ranges: dict[int, List[int]] = {}

    def __init__(self, n):
        self.n = n
        self.ranges[self.n] = [0, 0] # Will always be overridden 
        
    def __missing__(self, key): 
        if self.n == 1:
            val = MutableBool()
        else: 
            val = NGrid(self.n - 1)
        self[key] = val
            
        # Update minimum or maximum 
        if self.ranges[self.n][0] > key:
            self.ranges[self.n][0] = key
        if self.ranges[self.n][1] < key: 
            self.ranges[self.n][1] = key
        
        return val
        
    def life(self, steps): 
        """Simulate steps iterations of Conway's Game of Life."""
        for i in range(steps): 
            self.step()
    
    def step(self):
        """Simulate one iteration of Conway's Game of Life."""
        periods = list(map(lambda r: (r[0] - 1, r[1] + 2), self.ranges.values()))
        for coords in product(*[range(*period) for period in periods]):
            neighbors_on = sum(map(bool, self.neighbors(*coords)))
            if neighbors_on == 3: 
                nested_get(self, *coords).next = True 
            elif neighbors_on == 2 and not nested_get(self, *coords):
                nested_get(self, *coords).next = True 
        
        for coords in product(*[range(*period) for period in periods]):
            nested_get(self, *coords).update()
    
    def neighbors(self, *args, center=True) -> Generator[bool, None, None]:
        """Yield neighbors of coordinate at self[args[0]][args[1]][...]
        
        `center` tracks whether we've offset at least once during recursion. We 
        need to know this so we can ensure we never print out the element whose 
        neighbors we're searching for. 
        """
        if len(args) != self.n: 
            raise ValueError(f"Expected {self.n} integer coordinates, got {len(args)}.")
        if any(type(arg) != int for arg in args):
            raise ValueError("Expected integer coordinates.")
            
        first, *rest = args
        if self.n == 1:
            yield self[first - 1]
            yield self[first + 1]
            if not center:
                yield self[first]
        else: 
            yield from self[first - 1].neighbors(*rest, center=False)
            yield from self[first].neighbors(*rest, center=center)
            yield from self[first + 1].neighbors(*rest, center=False)

In [231]:
g = NGrid(3)

In [232]:
for y, l in enumerate(start): 
    for x, c in enumerate(l):
        if c == "#":
            g[0][y][x] = MutableBool(True)

In [233]:
g.life(6)

In [234]:
on = 0

In [235]:
def count(d): 
    if isinstance(d, Mapping):
        return sum(count(v) for k, v in d.items())
    else:
        if bool(d):
            return 1
        return 0

In [236]:
count(g)

590